In [80]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [89]:
eg_conn = psrcelmerpy.ElmerGeoConn()

# Load network shapefile
gdf = gpd.read_file(r'R:\e2projects_two\SoundCast\Inputs\dev\networks\2023\network_2023_v3\shapefiles\AM\AM_edges.shp')

In [90]:
len(gdf)

90799

In [91]:
# Load bike stress table, adapted from Lowry
# https://www1.coe.neu.edu/~pfurth/Furth%20papers/2016%20Prioritizing%20to%20improve%20low-stress%20network%20connectivity%20Lowry,%20Furth,%20Hadden-Loh.pdf

df_data = pd.read_csv('bike_stress_table.csv')

In [92]:
# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
gdf['SpeedLimit_new'] = gdf['ul2'].copy()
gdf.loc[gdf['ul2'] <= 25, 'SpeedLimit_new'] = 25
gdf.loc[gdf['ul2'] >= 35, 'SpeedLimit_new'] = 35

# Set maximum lanes at 6 and minimum at 2

gdf['lanes_new'] = gdf['lanes'].copy().astype('int')
gdf.loc[gdf['lanes'] >= 6, 'lanes_new'] = 6
gdf.loc[gdf['lanes'] <2, 'lanes_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityTy'] == 9, 'LocalStreet'] = 1
# gdf['FacilityType'].unique()

In [93]:
gdf['FacilityTy'] = gdf['FacilityTy'].astype('int')
gdf['FacilityTy'].unique()

# Remove centroid connectors and freeways
facility_type_lookup = {
    1:'Freeway',   # Interstate
    2:'Freeway',   # Other Freeway
    3:'Freeway', # Expressway
    4:'Ramp',
    5:'Arterial',    # Principal arterial
    6:'Arterial',    # Minor Arterial
    7:'Collector',    # Major Collector
    8:'Collector',    # Minor Collector
    9:'Collector',   # Local
    10:'Busway',
    11:'Non-Motor',
    12:'Light Rail',
    13:'Commuter Rail',
    15:'Ferry',
    16:'Passenger Only Ferry',
    17:'Connector',    # centroid connector
    18:'Connector',    # facility connector
    19:'HOV',    # HOV Only Freeway
    20:'HOV'    # HOV Flag
    }

gdf = gdf[gdf['FacilityTy'].isin([5,6,7,8,9,11])]

In [94]:
len(gdf)

61011

In [95]:
# Intersect with urban growth area
gdf_growth_area = eg_conn.read_geolayer('urban_growth_area')
gdf_growth_area = gdf_growth_area[['county_name','geometry']].to_crs('EPSG:2285')

gdf = gpd.sjoin(gdf, gdf_growth_area, how="left")

gdf['rural'] = 'not_rural'
gdf.loc[gdf['county_name'].isnull(),'rural'] = 'rural'

In [96]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Create new fields that use the bike facility and area type mapping
gdf['BikeFacility'] = gdf['bkfac'].map(bike_type_map)
gdf['AreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['SpeedLimit_new']>30, 'AreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood
# FIXME: should probably just be 1 lane in each direction
gdf.loc[(gdf['lanes_new']>2, 'AreaType')] = 'Other'

# If in rural area, treat paved shoulder as bike lanes
gdf.loc[((gdf['rural']=='rural')&(gdf['bkfac']==3)), 'BikeFacility'] = 'BikeLane'

# Merge to df_data to get stress values
gdf = gdf.merge(df_data, left_on=['SpeedLimit_new', 'lanes_new', 'BikeFacility','AreaType'], 
          right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left')

In [97]:
len(gdf)

61105

In [98]:
# add impact of slope
gdf.loc[(gdf['upslp'] > .02) & (gdf['upslp'] <= .04), 'Factor'] = gdf['Factor'] + 0.37
gdf.loc[(gdf['upslp'] > .04) & (gdf['upslp'] <= .06), 'Factor'] = gdf['Factor'] + 1.2
gdf.loc[(gdf['upslp'] > .06), 'Factor'] = gdf['Factor'] + 3.2

In [99]:
# Assign LTS based on stress factor
gdf.loc[gdf['Factor'] < 0.1, 'LTS'] = 1
gdf.loc[(gdf['Factor'] >= 0.1) & (gdf['Factor'] < 0.3), 'LTS'] = 2
gdf.loc[(gdf['Factor'] >= 0.3) & (gdf['Factor'] < 0.6), 'LTS'] = 3
gdf.loc[gdf['Factor'] >= 0.6, 'LTS'] = 4

In [100]:
# Create new column names because ArcGIS is a dinosaur that can't handle more than 10 character headers
gdf.rename(columns={'BikeFacility':'bkfac', 'SpeedLimit':'speed'}, inplace=True)

In [101]:
gdf['ij'] = gdf['i'].astype('str')+'-'+gdf['j'].astype('str')
gdf['ji'] = gdf['j'].astype('str')+'-'+gdf['i'].astype('str')

gdf['ji_exists'] = 0
gdf.loc[gdf['ji'].isin(gdf['ij']), 'ji_exists'] = 1

# Get a ji dataframe
gdf_ji = gdf[gdf['ji_exists']==1].copy()
gdf_ji.drop(['ij','geometry'], axis=1, inplace=True)
gdf_ji.rename(columns={'ji': 'ij'}, inplace=True)

# merge that to gdf

gdf_new = gdf.copy()
gdf_new = gdf_new[['ij','upslp',  'bkfac','Lanes', 'speed', 'Factor','LTS','geometry','PSRCEdgeID']].merge(gdf_ji[['ij','upslp', 'bkfac','Lanes', 'speed', 'Factor', 'LTS',]],
                                                                                      how='left', on='ij', suffixes=['_IJ','_JI'])
gdf_new.head()

,ij,upslp_IJ,bkfac_IJ,bkfac_IJ,Lanes_IJ,speed_IJ,Factor_IJ,LTS_IJ,geometry,PSRCEdgeID,upslp_JI,bkfac_JI,bkfac_JI,Lanes_JI,speed_JI,Factor_JI,LTS_JI
0,79117-79004,0.186476,0.0,NoBikeFacility,2,25,3.4,4.0,"LINESTRING (1274401.412 310281.765, 1274386.64...",131369.0,0.000000,0.0,NoBikeFacility,2.0,25.0,0.2,2.0
1,79004-72460,0.002948,0.0,NoBikeFacility,2,25,0.2,2.0,"LINESTRING (1274354.882 310296.588, 1274316.11...",131706.0,0.071124,0.0,NoBikeFacility,2.0,25.0,3.4,4.0
2,29618-29636,0.089987,1.0,BikeLane,2,35,3.7,4.0,"LINESTRING (1219375.412 78002.235, 1219408.000...",162329.0,0.000000,1.0,BikeLane,2.0,35.0,0.5,3.0
3,29587-29618,0.103284,1.0,BikeLane,2,35,3.7,4.0,"LINESTRING (1219343.176 77831.706, 1219356.176...",162314.0,0.000000,1.0,BikeLane,2.0,35.0,0.5,3.0
4,29636-29683,0.074871,1.0,BikeLane,2,35,3.7,4.0,"LINESTRING (1219408.000 78189.412, 1219453.588...",162438.0,0.000000,1.0,BikeLane,2.0,35.0,0.5,3.0


In [102]:
# for col in ['i','j']:
#     gdf_new[col] = gdf_new[col].astype('int32')
# gdf_new.to_file('2023_network_bike_ped_stress_psrc.shp')

In [103]:
len(gdf)

61105

In [110]:
# Merge with WSDOT and ped notebooks (created in separate notebooks)
gdf_ped = gpd.read_file('2023_ped_stress_index.shp')
gdf_wsdot = gpd.read_file('2023_network_bike_stress_wsdot.shp')

In [111]:
gdf_merge = gdf_new.merge(gdf_wsdot[['ij','BTLS_IJ','BTLS_JI']], on='ij', how='left')
gdf_merge = gdf_merge.merge(gdf_ped[['ij','PTLS_IJ','PTLS_JI']], on='ij', how='left')

In [112]:
gdf_merge.rename(columns={'LTS_IJ':'LTS_IJ_psrc', 'LTS_JI':'LTS_JI_psrc',
                          'BTLS_IJ': 'LTS_IJ_wsdot', 'BTLS_JI': 'LTS_JI_wsdot'}, inplace=True)

In [114]:
# gdf_merge.to_file('2023_bike_ped_stress.shp')
gdf_merge.to_csv(r'T:\2024October\brice\2023_bike_ped_stress_10_23_24.csv')